In [1]:
import pandas as pd
import numpy as np
import re
import json
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from pprint import pprint
import folium
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Métricas
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import pickle



In [2]:
class airbnb_city:
    
    def __init__(self, csvs, city_names):
            
            self.l_dfs = list()
            
            for enum, dataset in enumerate(csvs):
                
                self.l_dfs.append(pd.read_csv(dataset))
                
                self.l_dfs[enum].drop("source", axis = 1, inplace = True)
                
                self.l_dfs[enum]["city"] = city_names[enum].lower()
        
            self.df = pd.concat(self.l_dfs)
            
            print("Instance created!")
            
    def return_initial_df(self):
    
        return self.df
    
    def display__initial_df(self):
    
        display(self.df)

    def clean_tested_columns(self):
        
        """
        Sets predefined columns, transforms price to a float column and separates bathroom_text 
        into 3 different categories, private, shared and unknown.
        """
        
        # Sets predefined columns
        
        tested_cols = ['neighbourhood_cleansed', 'city',
                       'room_type', 'accommodates', 'availability_365',
                       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
                       'minimum_nights', 'maximum_nights',
                       'number_of_reviews', 'reviews_per_month', 'host_total_listings_count']
        
        self.df_cleaned = self.df[tested_cols]
        
        # Transforms price to a float column
        
        self.df_cleaned["price"] = self.df_cleaned["price"].apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(x) else x).values
            
        # Get numbers out of bathroom_text columns
        
        self.df_cleaned = self.df_cleaned[self.df_cleaned["bathrooms_text"].isnull() == False]

        l_nums = [re.findall(r'\d+',i) for i in self.df_cleaned["bathrooms_text"].values]

        l_nums_completed = []

        for i in l_nums:

            if len(i) > 1:

                l_nums_completed.append('.'.join(i))

            elif len(i) == 0:

                l_nums_completed.append('0')

            else:

                l_nums_completed.append(i[0])
                
        # Create two different columns replacing bathroom_text
        
        self.df_cleaned["bathrooms_text"] = l_nums_completed

        self.df_cleaned["bathrooms_text"] = self.df_cleaned["bathrooms_text"].astype("float64")
        
        # Amenities
        
        l_amenities_cleaned, l_amenities_sublisted = list(),list()
        
        for i in self.df_cleaned["amenities"]:

            l_amenities_sublisted.append(json.loads(i))

            for j in json.loads(i):

                l_amenities_cleaned.append(j)

        l_values = list(Counter(l_amenities_cleaned).values())
        l_keys = list(Counter(l_amenities_cleaned).keys())

        l_valuable_amenities, l_not_relevant_amenities = [], []

        for i in l_values:

            if i > 10000:

                l_valuable_amenities.append(l_keys[l_values.index(i)])

            else:

                l_not_relevant_amenities.append(l_keys[l_values.index(i)])


        l_popular_inter, l_exclusive_inter, l_binary_popular, l_binary_exclusive = [],[],[],[]

        popular_amenities = np.array(list(set(l_valuable_amenities)))
        exclusive_amenities = np.array(list(set(l_not_relevant_amenities)))

        for amenities in l_amenities_sublisted:

            l_popular_inter = np.intersect1d(ar1=amenities, ar2= popular_amenities)
            l_exclusive_inter = np.intersect1d(ar1=amenities, ar2= exclusive_amenities)

            if len(l_popular_inter) > len(amenities)/2:

                l_binary_popular.append(1)

            else:

                l_binary_popular.append(0)

            if len(l_exclusive_inter) > len(amenities)/2:

                l_binary_exclusive.append(1)

            else:

                l_binary_exclusive.append(0)

        self.df_cleaned["popular_amenities"] = l_binary_popular
        self.df_cleaned["exclusive_amenities"] = l_binary_exclusive
        self.df_cleaned.drop("amenities", axis = 1, inplace = True)
        
        
###################################################################################################################        
#         l_amenities_cleaned = list()
        
#         for i in self.df_cleaned["amenities"]:

#             l_amenities_cleaned.append(json.loads(i))

#         # Most relevant amenities, detailed analysis in the EDA file

#         l_amenities_valuables = ['Long term stays allowed','Cooking basics','Dishes and silverware','Essentials','Coffee maker','Hair dryer','Microwave','Refrigerator','Heating','Air conditioning']

#         for j in l_amenities_valuables:

#             self.df_cleaned[j] = [1 if j in i else 0 for i in l_amenities_cleaned]

#         self.df_cleaned.drop("amenities", axis =1, inplace=True)
        
#         self.df_cleaned.dropna(inplace = True)

###################################################################################################################
        # Room type
        
        self.df_cleaned = self.df_cleaned[self.df_cleaned["room_type"] != "Hotel room"]
        self.df_cleaned = pd.concat([self.df_cleaned, pd.get_dummies(data = self.df_cleaned["room_type"])], axis = 1).drop("room_type", axis = 1)
        
    def return_cleaned(self):
        
        return self.df_cleaned
    
    def display_cleaned(self):
        
        display(self.df_cleaned)
    
    def remove_outliers(self, accommodates, bathrooms_min, bathrooms_max, bedrooms, beds_min, beds_max, minimum_nights,
                       maximum_nights, nreviews, reviews_pmonth, price, htlc):

        self.df_cleaned = self.df_cleaned[self.df_cleaned["accommodates"] <= accommodates]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["bathrooms_text"].between(bathrooms_min, bathrooms_max)]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["bedrooms"] <= bedrooms]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["beds"].between(beds_min, beds_max)]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["minimum_nights"] <= minimum_nights]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["maximum_nights"] <= maximum_nights]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["number_of_reviews"] <= nreviews]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["reviews_per_month"] <= reviews_pmonth]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["price"] <= price]
        self.df_cleaned = self.df_cleaned[self.df_cleaned["host_total_listings_count"] <= htlc]

        return self.df_cleaned
    
    def display_outliers(self):
        
        for i in self.df_cleaned.columns:
    
            print(i)
            sns.kdeplot(self.df_cleaned[i])
            plt.show()

    def label_encoding(self):
        
        city_encoder = LabelEncoder()
        self.df_cleaned["city"] = city_encoder.fit_transform(self.df_cleaned["city"])
        neighbourhood_encoder = LabelEncoder()
        self.df_cleaned["neighbourhood_cleansed"] = neighbourhood_encoder.fit_transform(self.df_cleaned["neighbourhood_cleansed"])
        
        return self.df_cleaned
    
    def normalize(self):
        
        x_scaler = MinMaxScaler()
        self.df_cleaned[self.df_cleaned.drop("price", axis = 1).columns] = x_scaler.fit_transform(self.df_cleaned[self.df_cleaned.drop("price", axis = 1).columns])

        y_scaler = MinMaxScaler()
        self.df_cleaned["price"] = y_scaler.fit_transform(self.df_cleaned[["price"]]).flatten()
        
        return self.df_cleaned
    
    def tts(self):
        
        self.X = self.df_cleaned.drop(["price"], axis = 1)
        self.y = self.df_cleaned["price"]
                
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.2, random_state = 42)

        print(f"X_train: {self.X_train.shape} | y_train: {self.y_train.shape}")
        print(f"X_test: {self.X_test.shape} | y_test: {self.y_test.shape}")
    
    def train_model(self):
        
        #models = [LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor(),
        #          RandomForestRegressor(), SVR(), AdaBoostRegressor(), GradientBoostingRegressor()]
        
        
        models = [RandomForestRegressor()]
        
        metrics = list()
        
        for model in models:
            
            # fit
            
            model.fit(self.X_train, self.y_train)

            # predict
            
            self.yhat = model.predict(self.X_test)
            
            # metrics
            
            r2 = r2_score(self.y_test, self.yhat)
            mse = mean_squared_error(self.y_test, self.yhat)
        
            metrics.append([str(model), r2, mse, model])
            
        self.df_metrics = pd.DataFrame(data = metrics, columns = ["model_name", "r2", "mse", "model"])
        self.df_metrics.sort_values(by = "r2", ascending = False, inplace= True)
        
    def return_metrics(self):
        
        return self.df_metrics
    
    def display_metrics(self):
        
        display(self.df_metrics)
        
    def model_feature_importances(self, model):
        
        importances = np.argsort(model.feature_importances_)[::-1]
        d_importances = dict()
        
        for i in importances:

            d_importances[i] = [model.feature_importances_[i]*100, self.df_cleaned.drop("price", axis = 1).columns[i]]
            print(i, model.feature_importances_[i]*100, self.df_cleaned.drop("price", axis = 1).columns[i])
            
        return d_importances
    
    def grid_search_cv_tuning(self):
        
        model = RandomForestRegressor()
        
        params = {"n_estimators" : [i for i in range(100, 1001, 50)],
                  "max_depth"    : [8, 10, 12, 14, 16],
                  "max_features" : ["log2", "sqrt"]}

        scorers = {"r2", "neg_mean_squared_error"}

        grid_solver = GridSearchCV(estimator  = model, 
                                   param_grid = params, 
                                   scoring    = scorers,
                                   cv         = 10,
                                   refit      = "r2",
                                   n_jobs     = -1, 
                                   verbose    = 2)

        self.model_result = grid_solver.fit(X_train, y_train)
        
    def grid_search_cv_validation(self):
        
        l_validations = [self.model_result.best_estimator_,
                         self.model_result.cv_results_["mean_test_r2"].max(),
                         self.model_result.best_score_]
        self.df_validations = pd.DataFrame(data = l_validations, columns = ["Best Estimator",
                                                                            "Mean Test R**2",
                                                                            "Best Score"])
        return self.df_validations
    
    def final_trial_model(self):
        
        '''It trains the best model with the features recomended'''
        
        model = RandomForestRegressor(max_depth=16, max_features='sqrt', n_estimators=650, random_state = 42)
        model.fit(self.X_train, self.y_train)
        
        self.yhat = model.predict(self.X_test)
    
        return f"r**2 = {r2_score(self.y_test, self.yhat)}"
    
    def train_final_model(self, max_depth, max_features, n_estimators,random_state):
        
        '''Returns the definitive model'''
        
        self.X_def = self.df_cleaned.drop(["price"], axis = 1)
        self.y_def = self.df_cleaned["price"]
        
        model = RandomForestRegressor(max_depth = max_depth, max_features = max_features, n_estimators = n_estimators, random_state = random_state)
        model.fit(self.X_def, self.y_def)
        
        return model
    
    def predict(self, array):  
        
        '''Predicts the price given a cleaned array with te features needed'''
        
        price_predicted = y_scaler.inverse_transform([model.predict([array])])
        
        return price_predicted
    
    def save_model(self, name, ext, model):
    
        with open(f"{name}.{ext}", "wb") as file:
            pickle.dump(model)
            
    def load_model(self, name, ext):
        
        with open(f"{name}.{ext}", "rb") as file:
            self.model = pickle.load(file)
            
        
    

In [3]:
madrid = "datasets/madrid.csv"
rome = "datasets/rome.csv"
paris = "datasets/paris.csv"
oslo = "datasets/oslo.csv"
london = "datasets/london.csv"
geneva = "datasets/geneva.csv"
dublin = "datasets/dublin.csv"
barcelona = "datasets/barcelona.csv"
athens = "datasets/athens.csv"
amsterdam = "datasets/amsterdam.csv"

d_csvs, d_names = dict(), dict()

d_csvs["csvs1"] = [madrid, barcelona]
# d_csvs["csvs2"] = [london, paris]
# d_csvs["csvs3"] = [dublin, amsterdam]
# d_csvs["csvs4"] = [amsterdam, oslo]
# d_csvs["csvs5"] = [amsterdam, geneva]
# d_csvs["csvs6"] = [athens, dublin]
# d_csvs["csvs7"] = [paris, geneva, amsterdam]
d_csvs["csvs2"] = [dublin,geneva]
d_csvs["csvs3"] = [madrid, barcelona, dublin]
#d_csvs["csvs2"] = [madrid, barcelona, paris, london, amsterdam, rome, dublin, geneva, athens, oslo]

d_names["names1"] = ["madrid", "barcelona"]
d_names["names2"] = ["dublin", "geneva"]
d_names["names3"] = ["dublin", "amsterdam"]
# d_names["names4"] = ["amsterdam","oslo"]
# d_names["names5"] = ["amsterdam", "geneva"]
# d_names["names6"] = ["athens","dublin"]
# d_names["names7"] = ["paris", "geneva", "amsterdam"]
d_names["names3"] = ["madrid", "barcelona", "dublin"]
d_names["names4"] = ["madrid", "barcelona", "dublin"]
# d_names["names4"] = ["madrid", "barcelona", "london", "paris", "dublin", "rome", "amsterdam", "athens", "oslo", "geneva"]
# d_names["names2"] = ["madrid", "barcelona","paris", "london", "amsterdam", "rome","dublin","geneva","athens","oslo"]


In [4]:
d_dfs = dict()

In [5]:
%%time

for i in range(1,4):
    
    d_dfs[f"instance{i}"] = airbnb_city(d_csvs[f"csvs{i}"],d_names[f"names{i}"])

Instance created!
Instance created!
Instance created!
CPU times: user 2.37 s, sys: 213 ms, total: 2.58 s
Wall time: 2.59 s


In [6]:
d_dfs.values()

dict_values([<__main__.airbnb_city object at 0x7ff5f10336d0>, <__main__.airbnb_city object at 0x7ff5f0fb2ac0>, <__main__.airbnb_city object at 0x7ff600396ac0>])

In [7]:
%%time
for instance in d_dfs.values():
    
    instance.clean_tested_columns()

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_5686/4162281428.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df_cleaned["price"] = self.df_cleaned["price"].apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(x) else x).values
/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_5686/4162281428.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df_cleaned["price"] = self.df_cleaned["price"].apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(

CPU times: user 11 s, sys: 125 ms, total: 11.1 s
Wall time: 11.2 s


In [8]:
%%time
l_dfs = list()

for instance in d_dfs.values():
    
    l_dfs.append(instance.return_cleaned())

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 6.91 µs


In [9]:
%%time

for instance in d_dfs.values():
    
    instance.remove_outliers(accommodates=8,
                             bathrooms_min=1,
                             bathrooms_max=2,
                             bedrooms=4,
                             beds_min=1,
                             beds_max=5,
                             minimum_nights=30,
                             maximum_nights=70000,
                             nreviews=375,
                             reviews_pmonth=9,
                             price=350,
                             htlc=50000)

CPU times: user 43.9 ms, sys: 2.38 ms, total: 46.2 ms
Wall time: 47.7 ms


In [10]:
# d_dfs["instance2"].return_cleaned()

In [11]:
for instance in d_dfs.values():
    instance.label_encoding()

In [12]:
for instance in d_dfs.values():
    instance.normalize()

In [13]:
for instance in d_dfs.values():
    instance.tts()

X_train: (16786, 17) | y_train: (16786,)
X_test: (4197, 17) | y_test: (4197,)
X_train: (5707, 17) | y_train: (5707,)
X_test: (1427, 17) | y_test: (1427,)
X_train: (20743, 17) | y_train: (20743,)
X_test: (5186, 17) | y_test: (5186,)


In [14]:
%%time

for name, instance in d_dfs.items():
    
    instance.train_model()
    print(name)
    instance.display_metrics()

instance1


,model_name,r2,mse,model
0,RandomForestRegressor(),0.730779,0.013263,"(DecisionTreeRegressor(max_features='auto', ra..."


instance2


,model_name,r2,mse,model
0,RandomForestRegressor(),0.56836,0.015863,"(DecisionTreeRegressor(max_features='auto', ra..."


instance3


,model_name,r2,mse,model
0,RandomForestRegressor(),0.697426,0.014469,"(DecisionTreeRegressor(max_features='auto', ra..."


CPU times: user 14.9 s, sys: 103 ms, total: 15 s
Wall time: 15.1 s


In [18]:
d_dfs["instance1"].X_test

,neighbourhood_cleansed,city,accommodates,availability_365,bathrooms_text,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,reviews_per_month,host_total_listings_count,popular_amenities,exclusive_amenities,Entire home/apt,Private room,Shared room
15486,0.752577,1.0,0.428571,0.136986,0.0,0.333333,0.50,0.000000,0.000450,0.160428,0.847608,0.000000,1.0,0.0,1.0,0.0,0.0
3554,0.938144,0.0,0.857143,0.956164,0.0,1.000000,1.00,0.000000,0.101170,0.291444,0.170189,0.034839,1.0,0.0,1.0,0.0,0.0
796,0.639175,0.0,0.714286,0.780822,1.0,0.333333,1.00,0.034483,0.101170,0.016043,0.016685,0.082581,1.0,0.0,1.0,0.0,0.0
4641,0.479381,1.0,0.714286,0.758904,1.0,0.333333,0.75,0.034483,0.002430,0.438503,0.304783,0.000645,1.0,0.0,1.0,0.0,0.0
10389,0.556701,1.0,0.000000,0.556164,0.0,0.000000,0.00,1.000000,0.016112,0.016043,0.022247,0.000645,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15435,0.690722,1.0,0.142857,0.783562,1.0,0.000000,0.00,0.241379,0.005311,0.032086,0.192436,0.000000,0.0,1.0,0.0,1.0,0.0
17833,0.309278,1.0,0.000000,0.939726,0.0,0.666667,0.00,0.068966,0.032763,0.016043,0.252503,0.000645,0.0,1.0,0.0,1.0,0.0
8198,0.850515,0.0,0.428571,0.263014,0.0,0.333333,0.50,0.068966,0.101170,0.010695,0.014461,0.032903,1.0,0.0,1.0,0.0,0.0
15908,0.922680,0.0,0.428571,0.495890,0.0,0.333333,0.25,0.068966,0.101170,0.205882,0.199110,0.001935,1.0,0.0,0.0,1.0,0.0


In [15]:
d_dfs["instance2"].return_cleaned()

,neighbourhood_cleansed,city,accommodates,availability_365,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,reviews_per_month,host_total_listings_count,popular_amenities,exclusive_amenities,Entire home/apt,Private room,Shared room
0,0.400000,0.0,0.142857,0.284932,0.5,0.000000,0.25,0.176471,0.068966,1.000000,0.690860,0.204672,0.000434,0.0,1.0,0.0,1.0,0.0
1,0.400000,0.0,0.142857,0.301370,0.5,0.000000,0.00,0.167647,0.068966,0.017794,0.561828,0.169077,0.000434,0.0,1.0,0.0,1.0,0.0
2,0.422222,0.0,0.000000,0.794521,1.0,0.000000,0.00,0.102941,0.034483,0.017794,0.951613,0.308120,0.001301,0.0,1.0,0.0,1.0,0.0
3,0.400000,0.0,0.714286,0.202740,0.5,0.333333,0.75,0.205882,0.000000,0.009786,0.873656,0.408231,0.000434,0.0,1.0,0.0,1.0,0.0
4,0.422222,0.0,0.571429,0.084932,0.0,0.000000,0.50,0.708824,0.068966,0.323843,0.169355,0.052280,0.001735,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3363,0.066667,1.0,0.142857,0.082192,0.0,0.000000,0.00,0.058824,0.000000,1.000000,0.091398,0.102336,0.000434,0.0,1.0,0.0,1.0,0.0
3364,0.844444,1.0,0.142857,0.586301,0.0,0.000000,0.25,0.111765,0.000000,1.000000,0.002688,0.035595,0.000000,0.0,1.0,0.0,1.0,0.0
3365,0.488889,1.0,0.285714,0.243836,0.0,0.000000,0.25,0.152941,1.000000,1.000000,0.013441,0.011123,0.000434,0.0,1.0,1.0,0.0,0.0
3367,0.777778,1.0,0.142857,0.895890,0.0,0.000000,0.00,0.111765,0.206897,0.079181,0.021505,0.025584,0.000867,0.0,1.0,1.0,0.0,0.0


In [16]:
%%time
d_scores_ft = dict()

for name, instance in d_dfs.items():
    
    d_scores_ft[name] = instance.final_trial_model()
    

CPU times: user 24.8 s, sys: 412 ms, total: 25.2 s
Wall time: 25.4 s


In [17]:
d_scores_ft

{'instance1': 'r**2 = 0.7240094612897725',
 'instance2': 'r**2 = 0.5858164527528943',
 'instance3': 'r**2 = 0.6912471370709119'}